# Regresion Logistica: Deteccion de SPAM

En este ejercico se muestran los fundamentos de la regresion logistica, planteando uno de los primeros problemas que fueron solucionados mediante el uso ded tecnicas de Machine Learning: La deteccion de SPAM


##  Enunciado del ejercicio.
Se propone la construccion de un sistema de aprendizaje automatico capaz de predecir si un correo determinado se corresponde con un correo SPAM o no, para ello se utilizara el siguente DatSet:

##### [2007_TE _Public_Spam_Corpus (https://plg.uwaterloo.ca/~gvcormac/treccorpus07/)]
The corpus trec07p contains 75,419 messages:

    25220 ham
    50199 spam

These messages constitute all the messages delivered to a particular
server between these dates:

    Sun, 8 Apr 2007 13:07:21 -0400
    Fri, 6 Jul 2007 07:04:53 -0400

In [2]:
# En esta clase se facilita el procesamiento de correos electronicos 

# que poseen codigo html.
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.fed = []

    def handle_data(self, d):
        self.fed.append(d)

    def get_data(self):
        return ''.join(self.fed)

In [3]:
# Esta funcion se encarga de eliminar los tags HTML
# que se encunetren en el texto de los correos electronicos
def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [4]:
# Ejemplo de eliminacion de los tads HTML de un texto
t = '<tr><td align="left"><ahref="../../issues/51/16.html#article">Phrack world News </a><td>'
strip_tags(t)

'Phrack world News '

Ademas de eliminar los posiblrs tags html que se encuentran en el correo electronico deben realizarse otras acciones para evitar que los mensajes contengan ruido inecesario. Entre ellas se encuentra la eliminacion de signos de puntuacion, eliminancion de los posibles campos de correo electronico que no sean relevantes o eliminacion de los afijos de una palabra manteniendo unicamente la raiz de la misma(stemming). La clase que se muestra a continuacion realiza estas transformaciones.

In [10]:
import email
import string
import nltk

class Parser:
    def __init__(self):
        self.stemmer = nltl.PorterStemmer()
        self.stopwords = set(nltk.corpus.stopwords.words('english'))
        self.puntuaction = list(string.punctuation)

    def parse(self, email_path):
        """Parse an email."""
        with open(email_path, errors = 'ignore') as e:
            msg = email.message_from_file(e)
        return None if not msg else self.get_email_content(msg)

    def get_email_content(self, msg):
        """Extract the email content."""
        subject = self.tokenize(msg['Subject']) if msg['Subject'] else []
        body = self.get_email_body(msg.get_payload(),
                                  msg.get_content_type())
        content_type = msg.get_content_type()
        # Return the content of the email
        return {"subject": subject,
               "body": body,
               "content_type": content_type}

    def get_email_body(self, payload, content_type):
        """Extract the body of the email."""
        body = []
        if type(payload) is str and content_type == 'text/plain':
            return self.tokenize(payload)
        elif type(payload) is str and content_type == 'text/html':
            return self.tokenize(strip_tags(payload))
        elif type(payload) is list:
            for p in payload:
                body += self.get_email_body(p.get_payload(), 
                                           p.get_content_type())
        return body

    def tokenize(self, txt):
        """Transform a text string in tokens. Perform two main actons,
        clean the puntuaction symbols and do stemming of the text"""
        for c in self.puntuaction:
            text = text.replace(c, "")
        text = text.replace("\t", "")
        text = text.replace("\n", "")
        tokens = list(filter(None, text.split("")))
        # Stremming of the tokens
        return [self.stremmer.stem(w) for w in tokens if w not in self.stopword]

Lectura de un correo en formato .raw

In [15]:
inmail = open("datasets/datasets/trec07p/data/inmail.1").read()
print(inmail)

From RickyAmes@aol.com  Sun Apr  8 13:07:32 2007
Return-Path: <RickyAmes@aol.com>
Received: from 129.97.78.23 ([211.202.101.74])
	by speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;
	Sun, 8 Apr 2007 13:07:21 -0400
Received: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100
Message-ID: <WYADCKPDFWWTWTXNFVUE@yahoo.com>
From: "Tomas Jacobs" <RickyAmes@aol.com>
Reply-To: "Tomas Jacobs" <RickyAmes@aol.com>
To: the00@speedy.uwaterloo.ca
Subject: Generic Cialis, branded quality@ 
Date: Sun, 08 Apr 2007 21:00:48 +0300
X-Mailer: Microsoft Outlook Express 6.00.2600.0000
MIME-Version: 1.0
Content-Type: multipart/alternative;
	boundary="--8896484051606557286"
X-Priority: 3
X-MSMail-Priority: Normal
Status: RO
Content-Length: 988
Lines: 24

----8896484051606557286
Content-Type: text/html;
Content-Transfer-Encoding: 7Bit

<html>
<body bgcolor="#ffffff">
<div style="border-color: #00FFFF; border-right-width: 0px; border-bottom-width: 0px; margin-bottom: 0px;" align="